In [2]:
import pandas as pd
import numpy as np
import re
import pyarrow.parquet as pq
import uuid

# GOOGLE MAPS

## Carpeta "metadata-sitios"

Creo variables con el path de cada archivo

In [3]:
path1 = "metadata-sitios/1.json"
path2= "metadata-sitios/2.json"
path3 = "metadata-sitios/3.json"
path4= "metadata-sitios/4.json"
path5 = "metadata-sitios/5.json"
path6= "metadata-sitios/6.json"
path7 = "metadata-sitios/7.json"
path8= "metadata-sitios/8.json"
path9 = "metadata-sitios/9.json"
path10= "metadata-sitios/10.json"
path11 = "metadata-sitios/11.json"


Extraigo primero el archivo *1.json* para analizarlo
 

In [5]:
# Extraigo
gm_metadata_1= pd.read_json(path1, lines=True)

# Reviso
print(gm_metadata_1.info())
gm_metadata_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB
None


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


Reviso las variables numéricas:

In [6]:
gm_metadata_1.describe()

,latitude,longitude,avg_rating,num_of_reviews
count,275001.000000,275001.000000,275001.000000,275001.000000
mean,37.490112,-92.274708,4.307216,19.722750
std,5.405780,16.414617,0.790101,37.642092
min,-1.766059,-176.644648,1.000000,1.000000
25%,33.759405,-98.924254,4.000000,4.000000
50%,38.411790,-87.609055,4.500000,8.000000
75%,41.320476,-80.252418,5.000000,18.000000
max,70.398544,180.000000,5.000000,2114.000000


De lo anterior:
* Las variables numéricas están con el tipo de dato correcto.
* Hay columnas inncesarias en relación a nuestro objetivo: *description*, *hours*, *price*, *url*, *relative_results*, *state*.
* En las columnas más importantes de acuerdo a nuestro objetivo, como *gmap_id*, *latitude*, *longitude*, *category*, *avg_rating* y *MISC* hay casi cero nulos o muy pocos, excepto en la última columna *MISC*, en la que hay 80 mil nulos aproximadamente. Conviene revisarnuevamente los nulos después de filtrar el dataframe con los datos de interés. 
* Ya que el foco está en los estados California, Texas y Florida, ya que son los que tienen mayor población latina, se debe obtener una columna *state* a partir de los datos de *address* que contenga solo el iso del estado : CA, FL, TX, y filtrar el df a partir de esto.
* Se debe identificcar aquellos establecimientos que son restaurantes o afines a partir de la columna *category* y filtrar el dataframe.
* Ya con el dataframe filtrado, se debe revisar los duplicados.

Primero se obtendrá el estado para el filtrado:

In [9]:
# Reemplazar la columna "state", que tiene información irrelevante, y normalizar por el ISO del estado solo los que tienen CA, FL y TX
gm_metadata_1['state'] = gm_metadata_1['address'].str.upper()
gm_metadata_1['state'] = gm_metadata_1['state'].str.replace('FLORIDA', 'FL')
gm_metadata_1['state'] = gm_metadata_1['state'].str.replace('CALIFORNIA', 'CA')
gm_metadata_1['state'] = gm_metadata_1['state'].str.replace('TEXAS', 'TX')       

In [11]:
# Revisar los registros que contienen el ISO o el nombre de los estados y reemplazar por el ISO correspondiente
gm_metadata_1['state'] = gm_metadata_1.apply(lambda row: 'FL' if row['state'] is not None and ('FL' in row['state'] or 'FLORIDA' in row['state']) else row['state'], axis=1)
gm_metadata_1['state'] = gm_metadata_1.apply(lambda row: 'CA' if row['state'] is not None and ('CA' in row['state'] or 'CALIFORNIA' in row['state']) else row['state'], axis=1)
gm_metadata_1['state'] = gm_metadata_1.apply(lambda row: 'TX' if row['state'] is not None and ('TX' in row['state'] or 'TEXAS' in row['state']) else row['state'], axis=1)
        
# Reviso:
gm_metadata_1.state.head()

0    PORTER PHARMACY, 129 N SECOND ST, COCHRAN, GA ...
1                                                   CA
2                                                   CA
3                                                   CA
4                                                   CA
Name: state, dtype: object

In [15]:
# Filtrar por los estados determinados
gm_metadata_1 = gm_metadata_1[gm_metadata_1['state'].isin(['CA', 'FL', 'TX'])]

# Reviso:
gm_metadata_1.state.unique()

array(['CA', 'FL', 'TX'], dtype=object)

Ahora, se filtrará por restaurantes:

In [16]:
# Obtengo una lista de palabras clave a buscar
palabras_clave = ['restaurant', 'cafe', 'cafeteria', 'fast food', 'fastfood', 'mexican food', 'hamburger']

# Función para buscar coincidencias de palabras clave en la lista de strings
def buscar_coincidencias(lista):
    if lista is None:
        return False
    else:
        texto = ' '.join(lista).lower()
        for palabra in palabras_clave:
            if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                return True
        return False

In [17]:
# Verifico si alguna palabra clave está presente en las listas de strings de la columna 'category'
gm_metadata_1['contains_keyword'] = gm_metadata_1['category'].apply(buscar_coincidencias)      
                
# Filtro el DataFrame para obtener solo las filas que contienen palabras clave (los que sean TRUE)
gm_metadata_1 = gm_metadata_1[gm_metadata_1['contains_keyword']]

In [18]:
# Reviso
gm_metadata_1.category.head(10)

2           [Korean restaurant]
6                  [Restaurant]
114                [Restaurant]
115         [Buffet restaurant]
123                [Restaurant]
269           [Taco restaurant]
271                [Restaurant]
484                [Restaurant]
566    [Health food restaurant]
578        [Seafood restaurant]
Name: category, dtype: object

Elimino las columnas innecesarias:

In [19]:
# Elimino la columna 'contains_keyword' si ya no es necesaria
columns = ["description", "hours", "price", "url", "relative_results", "contains_keyword"]
gm_metadata_1.drop(columns, axis=1, inplace=True)


Reviso duplicados:

In [20]:
# Reviso los duplicados de gmap_id:
gm_metadata_1.gmap_id.duplicated().sum()

517

Elimino los duplicados a partir de la columna *gmap_id*:

In [24]:
gm_metadata_1.drop_duplicates(subset="gmap_id", inplace=True)

Reviso los duplicados en general:

In [25]:
# Exceptúo las columnas "category" y "MISC" para poder aplicar el método duplicated() , ya que contienen listas y diccionarios:
gm_metadata_1.drop(["category", "MISC"], axis=1).duplicated().sum()

0

Ya no hay duplicados. Por último, reviso los nulos:

In [26]:
gm_metadata_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4901 entries, 2 to 274964
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            4901 non-null   object 
 1   address         4901 non-null   object 
 2   gmap_id         4901 non-null   object 
 3   latitude        4901 non-null   float64
 4   longitude       4901 non-null   float64
 5   category        4901 non-null   object 
 6   avg_rating      4901 non-null   float64
 7   num_of_reviews  4901 non-null   int64  
 8   MISC            4858 non-null   object 
 9   state           4901 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 421.2+ KB


Solo la columna *MISC* presenta nulos. Ya que esta columna contiene información relevante y son pocos nulos, imputo su valor por "No data":

In [27]:
gm_metadata_1.MISC.fillna("No data", inplace= True)

# Reviso:
gm_metadata_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4901 entries, 2 to 274964
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            4901 non-null   object 
 1   address         4901 non-null   object 
 2   gmap_id         4901 non-null   object 
 3   latitude        4901 non-null   float64
 4   longitude       4901 non-null   float64
 5   category        4901 non-null   object 
 6   avg_rating      4901 non-null   float64
 7   num_of_reviews  4901 non-null   int64  
 8   MISC            4901 non-null   object 
 9   state           4901 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 421.2+ KB


Reindexo:

In [28]:
gm_metadata_1.reset_index(drop=True, inplace=True)

# Reviso:
gm_metadata_1.head(2)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,MISC,state
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4,18,"{'Service options': ['Takeout', 'Dine-in', 'De...",CA
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,[Restaurant],4.5,18,"{'Service options': ['Outdoor seating', 'Curbs...",CA


Ahora que todas las transformaciones están realizadas, se debe separar en las tablas necesarias para el MER:

* Creo una tabla para los estados:

In [38]:
ids = [1,2,3] 
iso= gm_metadata_1["state"].unique()
names= ['California','Florida', 'Texas']

states = {"state_id": ids , "iso": iso , 'name': names}
states = pd.DataFrame(states)
# Reviso:
states

,state_id,iso,name
0,1,CA,California
1,2,FL,Florida
2,3,TX,Texas


* Creo una nueva columna en gm_metadata_1 con el *state_id* y elimino la columna *state*:

In [42]:
# Obtengo los valores de state_id en el df gm_metadata_1:
gm_metadata_1['state_id'] = gm_metadata_1['state'].map(states.set_index('iso')['state_id'])

# Elimino la columna "state":
gm_metadata_1.drop("state", axis=1, inplace=True)

# Reviso:
gm_metadata_1.head(2)

,name,address,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,MISC,state_id
0,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4,18,"{'Service options': ['Takeout', 'Dine-in', 'De...",1
1,Vons Chicken,"Vons Chicken, 12740 La Mirada Blvd, La Mirada,...",0x80dd2b4c8555edb7:0xfc33d65c4bdbef42,33.916402,-118.010855,[Restaurant],4.5,18,"{'Service options': ['Outdoor seating', 'Curbs...",1


* Finalmente, cambio el orden de las columnas:

In [45]:
gm_metadata_1.columns.values

array(['name', 'address', 'gmap_id', 'latitude', 'longitude', 'category',
       'avg_rating', 'num_of_reviews', 'MISC', 'state_id'], dtype=object)

In [48]:
# Obtengo una lista con las columnas en el orden deseado:
columns= ['gmap_id', 'name', 'address', 'category', 'avg_rating', 
           'num_of_reviews', 'MISC','latitude', 'longitude','state_id']

# Cambio el orden de las columnas en el DataFrame
gm_metadata_1 = gm_metadata_1[columns]

# Reviso:
gm_metadata_1.columns.values

array(['gmap_id', 'name', 'address', 'category', 'avg_rating',
       'num_of_reviews', 'MISC', 'latitude', 'longitude', 'state_id'],
      dtype=object)

### 1. FUNCIÓN process_gm_metadata_files()

Ya todas las transformaciones están listas. Creo una función a partir de ellas para automatizarlo:

In [54]:

def process_gm_metadata_files(paths):
    result_dataframes = []
# EXTRACCIÓN
    for path in paths:
        # Leer el archivo JSON y convertirlo en DataFrame
        df = pd.read_json(path, lines=True)
        
# TRANSFORMACIÓN
        # Reemplaza la columna "state" y normalizar por el ISO del estado
        df['state'] = df['address'].str.upper()
        df['state'] = df['state'].str.replace('FLORIDA', 'FL')
        df['state'] = df['state'].str.replace('CALIFORNIA', 'CA')
        df['state'] = df['state'].str.replace('TEXAS', 'TX')

        # Revisar los registros que contienen el ISO o el nombre de los estados y reemplazar por el ISO correspondiente
        df['state'] = df.apply(lambda row: 'FL' if row['state'] is not None and ('FL' in row['state'] or 'FLORIDA' in row['state']) else row['state'], axis=1)
        df['state'] = df.apply(lambda row: 'CA' if row['state'] is not None and ('CA' in row['state'] or 'CALIFORNIA' in row['state']) else row['state'], axis=1)
        df['state'] = df.apply(lambda row: 'TX' if row['state'] is not None and ('TX' in row['state'] or 'TEXAS' in row['state']) else row['state'], axis=1)
        
        #Filtrar por los estados determinados
        df = df[df['state'].isin(['CA', 'FL', 'TX'])]
        
        # Lista de palabras clave a buscar
        palabras_clave = ['restaurant', 'cafe', 'cafeteria', 'fast food', 'fastfood', 'mexican food', 'hamburger']

        # Función para buscar coincidencias de palabras clave en la lista de strings
        def buscar_coincidencias(lista):
            if lista is None:
                return False
            else:
                texto = ' '.join(lista).lower()
                for palabra in palabras_clave:
                    if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                        return True
                return False

        # Verificar si alguna palabra clave está presente en las listas de strings de la columna 'category'
        df['contains_keyword'] = df['category'].apply(buscar_coincidencias)      
                
        # Filtrar el DataFrame para obtener solo las filas que contienen palabras clave (los que sean TRUE)
        df = df[df['contains_keyword']]

        # Eliminar la columna 'contains_keyword' si ya no es necesaria
        df.drop('contains_keyword', axis=1, inplace=True)

        # Borrar duplicados de gmap_id:
        df.drop_duplicates(subset="gmap_id", inplace=True)
        
        # Eliminar columnas innecesarias:
        columns = ["description", "hours", "price", "url", "relative_results"]
        df.drop(columns, axis=1, inplace=True)
        
        # Agregar el DataFrame final a la lista de resultados
        result_dataframes.append(df)
              
    # Une los DataFrames verticalmente
    merged_dataframe = pd.concat(result_dataframes, axis=0)
    
    # Imputo nulos en la columna "MISC"
    merged_dataframe.MISC.fillna("No data", inplace= True)

    # Reiniciar los índices 
    merged_dataframe.reset_index(drop=True, inplace=True)
    
    # Obtengo una nueva tabla states:
    states = {"state_id": [1,2,3] , "iso": merged_dataframe["state"].unique() , 'name': ['California','Florida', 'Texas']}
    states = pd.DataFrame(states)
    
    # Obtengo los valores de state_id en el df gm_metadata_1:
    merged_dataframe['state_id'] = merged_dataframe['state'].map(states.set_index('iso')['state_id'])

    # Elimino la columna "state":
    merged_dataframe.drop("state", axis=1, inplace=True)
    
    # Reordeno las columnas:
    columns= ['gmap_id', 'name', 'address', 'category', 'avg_rating', 
           'num_of_reviews', 'MISC','latitude', 'longitude','state_id']

    merged_dataframe = merged_dataframe[columns]
    

# CARGA    
    merged_dataframe.to_csv('POST_ETL_DATASETS/Google_Maps_Data/gm_restaurants.csv', index=False, encoding="utf-8")
    states.to_csv('POST_ETL_DATASETS/Google_Maps_Data/states.csv', index=False, encoding="utf-8")
    print('Archivos csv guardados con éxito.')
    
    return merged_dataframe, states



Creo una lista con los paths de cada archivo:

In [55]:
# Lista de rutas de archivos JSON
paths = [ path1,path2,path3,
          path4,path5,path6,
          path7,path8,path9,
          path10,path11 ]

In [56]:
gm_restaurants, states= process_gm_metadata_files(paths)

Archivos csv guardados con éxito.


In [60]:
# Reviso:
gm_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100287 entries, 0 to 100286
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   gmap_id         100287 non-null  object 
 1   name            100287 non-null  object 
 2   address         100287 non-null  object 
 3   category        100287 non-null  object 
 4   avg_rating      100287 non-null  float64
 5   num_of_reviews  100287 non-null  int64  
 6   MISC            100287 non-null  object 
 7   latitude        100287 non-null  float64
 8   longitude       100287 non-null  float64
 9   state_id        100287 non-null  int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 7.7+ MB


In [59]:
# Reviso:
states

,state_id,iso,name
0,1,CA,California
1,2,FL,Florida
2,3,TX,Texas


## Carpeta "reviews-estados"

Ya que nuestro objetivo general se centra en los estados con mayor pobalción de latinos, el alcance será solo en los estados California , Florida y Texas.



### California

* Creo variables con el paths de cada archivo

In [7]:
cal1 = "reviews-estados/review-California/1.json"
cal2 = "reviews-estados/review-California/2.json"
cal3 = "reviews-estados/review-California/3.json"
cal4 = "reviews-estados/review-California/4.json"
cal5 = "reviews-estados/review-California/5.json"
cal6 = "reviews-estados/review-California/6.json"
cal7 = "reviews-estados/review-California/7.json"
cal8 = "reviews-estados/review-California/8.json"
cal9 = "reviews-estados/review-California/9.json"
cal10 = "reviews-estados/review-California/10.json"
cal11 = "reviews-estados/review-California/11.json"
cal12 = "reviews-estados/review-California/12.json"
cal13 = "reviews-estados/review-California/13.json"
cal14 = "reviews-estados/review-California/14.json"
cal15 ="reviews-estados/review-California/15.json"
cal16 = "reviews-estados/review-California/16.json"
cal17 = "reviews-estados/review-California/17.json"
cal18 = "reviews-estados/review-California/18.json"


* Obtengo el primer dataset en un dataframe

In [8]:
cal_1 = pd.read_json(cal1, lines= True)


*  Genero una copia del df para poder explorarlo sin modificar nada

In [9]:
cal_1_copy = cal_1.copy()
cal_1_copy.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,1609909927056,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,1612849648663,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,1583643882296,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,1551938216355,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.005808e+20,Mario Marzouk,1494910901933,5,Great food and prices the portions are large,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


* Reviso su información

In [65]:
cal_1_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     89135 non-null   object 
 5   pics     5632 non-null    object 
 6   resp     16935 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB


* Reviso las variables numéricas:

In [10]:
cal_1_copy.describe()

,user_id,time,rating
count,1.500000e+05,1.500000e+05,150000.00000
mean,1.096641e+20,1.553771e+12,4.31154
std,5.140433e+18,5.169336e+10,1.21477
min,1.000002e+20,1.081296e+12,1.00000
25%,1.055420e+20,1.521335e+12,4.00000
50%,1.096182e+20,1.558723e+12,5.00000
75%,1.140358e+20,1.595102e+12,5.00000
max,1.184467e+20,1.631149e+12,5.00000


De lo anterior:
* La columna *time* debe ser pasada a formato de fecha ya que está en valor numérico.
* Hay muchos nulos en la columna *text* y *resp*; esto conviene revisarlo nuevamente después de unir los datasets de reviews  pertenecientes a cada estado para confirmar si representará un problema.
* Filtrar a partir del año 2018 ya que nos interesan los datos más actuales.
* Filtrar solo por restaurantes.
* Columnas innecesarias: *pics* y *name*.

Cambio el tipo de dato de la columna *time* a date_time:

In [66]:
# Especifico la unidad de tiempo en la están los registros (ms) y lo convierto a fecha:
cal_1_copy['time'] = pd.to_datetime(cal_1_copy['time'], unit='ms').dt.date

# Reviso el cambio:
cal_1_copy['time'].head()


0    2021-01-06
1    2021-02-09
2    2020-03-08
3    2019-03-07
4    2017-05-16
Name: time, dtype: object

In [67]:
# Verifico el tipo de dato:
type(cal_1_copy.time[0])

datetime.date

Reviso la fecha más antigua y más reciente

In [68]:
print(cal_1_copy['time'].min())
print(cal_1_copy['time'].max())

2004-04-07
2021-09-09


Reviso cúantos registros hay a partir del año 2018:

In [69]:
cal_1_copy[cal_1_copy['time'].apply(lambda x: x.year)>=2018].shape


(118152, 8)

Filtro a partir del año 2018

In [70]:
cal_1_copy=cal_1_copy[cal_1_copy['time'].apply(lambda x: x.year)>=2018]

Filtro solo los restaurantes

In [72]:
# Genero una lista de los gmap_id de los restaurantes del df gmap_metadata:
lista_id_rest= list(gm_restaurants['gmap_id'])


In [74]:
# Filtro los establecimientos de California a partir de la lista anterior:
cal_1_restaurantes = cal_1_copy[cal_1_copy['gmap_id'].isin(lista_id_rest)]

# Reviso el dataframe:
cal_1_restaurantes.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.089912e+20,Song Ro,2021-01-06,5,Love there korean rice cake.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.112903e+20,Rafa Robles,2021-02-09,5,Good very good,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.126404e+20,David Han,2020-03-08,4,They make Korean traditional food very properly.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.174403e+20,Anthony Kim,2019-03-07,5,Short ribs are very delicious.,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49
5,1.001857e+20,Ana Salazar,2019-01-18,5,This food is delicious 😁,None,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49


Elimino las columnas innecesarias:

In [75]:
cal_1_restaurantes = cal_1_restaurantes.drop(['name','pics'], axis=1)

# Reviso el dataframe:
cal_1_restaurantes.head()
print(cal_1_restaurantes.info())

<class 'pandas.core.frame.DataFrame'>
Index: 10370 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  10370 non-null  float64
 1   time     10370 non-null  object 
 2   rating   10370 non-null  int64  
 3   text     6150 non-null   object 
 4   resp     855 non-null    object 
 5   gmap_id  10370 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 567.1+ KB
None


Reviso duplicados:

In [76]:
duplicados=cal_1_restaurantes.drop("resp", axis=1).duplicated(keep=False)
cal_1_restaurantes[duplicados].sort_values('user_id', ascending=True)

,user_id,time,rating,text,resp,gmap_id
37192,1.000833e+20,2021-03-07,4,(Translated by Google) Obi\n\n(Original)\n오비,None,0x80c2b9518dc4f7e7:0xdfbf945567a04fe2
37191,1.000833e+20,2021-03-07,4,(Translated by Google) Obi\n\n(Original)\n오비,None,0x80c2b9518dc4f7e7:0xdfbf945567a04fe2
23464,1.001105e+20,2018-03-10,1,The place will mess up your order,None,0x80dbfd54e86940cf:0xf80b78d185db50aa
9521,1.001105e+20,2018-03-10,1,The place will mess up your order,None,0x80dbfd54e86940cf:0xf80b78d185db50aa
12361,1.001731e+20,2021-07-04,5,Nice staff,None,0x808f9f16784e6f09:0x8d47c5981115ff6b
...,...,...,...,...,...,...
3199,1.183516e+20,2020-12-13,4,None,None,0x80c2c7049ff0132d:0xf598d3badb3f6dc
26113,1.183546e+20,2020-02-08,1,"Awful service, ordered on doordash never got t...",None,0x808fc1eb8d12835d:0x29e8600a14d5caef
12170,1.183546e+20,2020-02-08,1,"Awful service, ordered on doordash never got t...",None,0x808fc1eb8d12835d:0x29e8600a14d5caef
4587,1.184250e+20,2019-10-17,2,None,None,0x808f7c1425bf6e9d:0xf5c6c8e213483c28


Elimino las filas duplicadas:

In [77]:
# Obtengo las filas duplicadas sin contar la columna "resp" ya que es un diccionario:
duplicados = cal_1_restaurantes.drop("resp", axis=1).duplicated(keep='first')
indices_duplicados = duplicados[duplicados].index

# Elimino las filas duplicadas del DataFrame original por índice:
cal_1_restaurantes.drop(indices_duplicados, inplace=True)

# Reviso:
cal_1_restaurantes.info()


<class 'pandas.core.frame.DataFrame'>
Index: 9739 entries, 0 to 149999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   user_id  9739 non-null   float64
 1   time     9739 non-null   object 
 2   rating   9739 non-null   int64  
 3   text     5731 non-null   object 
 4   resp     754 non-null    object 
 5   gmap_id  9739 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 532.6+ KB


Agrego una columna con el iso del estado:

In [83]:
cal_1_restaurantes['state_iso'] = "CA"
cal_1_restaurantes.head(2)

,user_id,time,rating,text,resp,gmap_id,state_iso
0,1.089912e+20,2021-01-06,5,Love there korean rice cake.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA
1,1.112903e+20,2021-02-09,5,Good very good,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA


Finalmente, genero una columna review_id:

In [89]:
cal_1_restaurantes['review_id'] = [uuid.uuid4() for _ in range(len(cal_1_restaurantes))]

cal_1_restaurantes.head()

,user_id,time,rating,text,resp,gmap_id,state_iso,review_id
0,1.089912e+20,2021-01-06,5,Love there korean rice cake.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA,2e3229dc-a699-4850-9558-9bc0bd03cecc
1,1.112903e+20,2021-02-09,5,Good very good,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA,a403a64f-bcde-4363-bbd7-b676e4848054
2,1.126404e+20,2020-03-08,4,They make Korean traditional food very properly.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA,6c9c8cce-6991-4910-90b7-5452d75e28de
3,1.174403e+20,2019-03-07,5,Short ribs are very delicious.,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA,8b7e0abd-4025-4306-b2de-9318b03de20c
5,1.001857e+20,2019-01-18,5,This food is delicious 😁,None,0x80c2c778e3b73d33:0xbdc58662a4a97d49,CA,7516a677-6aca-4015-80b4-e194ace1f54e


In [90]:
# Reviso:
cal_1_restaurantes.review_id.duplicated().sum()

0

Todas las transformaciones necesarias ya están realizadas. A partir de estas creo una función que permita automatizar el ETL de los reviews de cada estado (California, Florida y Texas).

### 2. FUNCIÓN process_gm_reviews_files()

In [102]:
def process_gm_reviews_files(paths, df_metadata=None):
    if df_metadata is None:
        df_metadata = pd.read_csv("POST_ETL_DATASETS/Google_Maps_Data/gm_restaurants.csv")
        
    states = pd.read_csv('POST_ETL_DATASETS/Google_Maps_Data/states.csv')
    
    result_dataframes = []
    
    for state_idx, state_paths in enumerate(paths):
        state_iso = states.loc[state_idx, 'iso']
        
        for path in state_paths:
            df = pd.read_json(path, lines=True)
            
            df['time'] = pd.to_datetime(df['time'], unit='ms').dt.date
            df = df[df['time'].apply(lambda x: x.year) >= 2018]
            
            lista_id_restaurantes = list(df_metadata['gmap_id'])
            df = df[df['gmap_id'].isin(lista_id_restaurantes)]
            
            df = df.drop(['name', 'pics'], axis=1)
            
            # Creo la columna 'state' 
            df['state_iso'] = state_iso
            
            # Cambio el nombre de la columna "time" a "date":
            df.rename(columns = {'time': 'date'}, inplace=True)
            
            # Agrego el df a la lista de dataframes:    
            result_dataframes.append(df)
     
    # Concatenar los DataFrames generados para CA, FL y TX    
    merged_dataframe = pd.concat(result_dataframes, axis=0)
    merged_dataframe.reset_index(drop=True, inplace=True)
    
     # Obtengo las filas duplicadas sin contar la columna "resp" ya que es un diccionario:
    duplicados = merged_dataframe.drop("resp", axis=1).duplicated(keep='first')
    indices_duplicados = duplicados[duplicados].index
    merged_dataframe.drop(indices_duplicados, inplace=True)
    merged_dataframe.reset_index(drop=True, inplace=True)
    
    # Generar una columna nueva con un identificador único para cada review:
    merged_dataframe['review_id'] = [uuid.uuid4() for _ in range(len(merged_dataframe))]
    
    # Ordeno columnas:
    columns = ['review_id', 'user_id', 'date', 'rating', 'text', 'resp', 'gmap_id', 'state_iso']
    merged_dataframe = merged_dataframe[columns]
    
    #CARGA
    load_path= "POST_ETL_DATASETS/Google_Maps_Data/gm_reviews.csv"
    merged_dataframe.to_csv(load_path, index=False, encoding="utf-8")
    print(f"Archivo CSV guardado con éxito")
    
    return merged_dataframe



Obtengo los paths:

In [104]:
# California
cal1 = "reviews-estados/review-California/1.json"
cal2 = "reviews-estados/review-California/2.json"
cal3 = "reviews-estados/review-California/3.json"
cal4 = "reviews-estados/review-California/4.json"
cal5 = "reviews-estados/review-California/5.json"
cal6 = "reviews-estados/review-California/6.json"
cal7 = "reviews-estados/review-California/7.json"
cal8 = "reviews-estados/review-California/8.json"
cal9 = "reviews-estados/review-California/9.json"
cal10 = "reviews-estados/review-California/10.json"
cal11 = "reviews-estados/review-California/11.json"
cal12 = "reviews-estados/review-California/12.json"
cal13 = "reviews-estados/review-California/13.json"
cal14 = "reviews-estados/review-California/14.json"
cal15 ="reviews-estados/review-California/15.json"
cal16 = "reviews-estados/review-California/16.json"
cal17 = "reviews-estados/review-California/17.json"
cal18 = "reviews-estados/review-California/18.json"

# Florida
florida1 ="reviews-estados/review-Florida/1.json"
florida2 ="reviews-estados/review-Florida/2.json"
florida3 ="reviews-estados/review-Florida/3.json"
florida4 ="reviews-estados/review-Florida/4.json"
florida5 ="reviews-estados/review-Florida/5.json"
florida6 ="reviews-estados/review-Florida/6.json"
florida7 ="reviews-estados/review-Florida/7.json"
florida8 ="reviews-estados/review-Florida/8.json"
florida9 ="reviews-estados/review-Florida/9.json"
florida10 ="reviews-estados/review-Florida/10.json"
florida11 ="reviews-estados/review-Florida/11.json"
florida12 ="reviews-estados/review-Florida/12.json"
florida13 ="reviews-estados/review-Florida/13.json"
florida14 ="reviews-estados/review-Florida/14.json"
florida15 ="reviews-estados/review-Florida/15.json"
florida16 ="reviews-estados/review-Florida/16.json"
florida17 ="reviews-estados/review-Florida/17.json"
florida18 ="reviews-estados/review-Florida/18.json"
florida19 ="reviews-estados/review-Florida/19.json"

# Texas
texas1 = "reviews-estados/review-Texas/1.json"
texas2 = "reviews-estados/review-Texas/2.json"
texas3 = "reviews-estados/review-Texas/3.json"
texas4 = "reviews-estados/review-Texas/4.json"
texas5 = "reviews-estados/review-Texas/5.json"
texas6 = "reviews-estados/review-Texas/6.json"
texas7 = "reviews-estados/review-Texas/7.json"
texas8 = "reviews-estados/review-Texas/8.json"
texas9 = "reviews-estados/review-Texas/9.json"
texas10 = "reviews-estados/review-Texas/10.json"
texas11 = "reviews-estados/review-Texas/11.json"
texas12 = "reviews-estados/review-Texas/12.json"
texas13 = "reviews-estados/review-Texas/13.json"
texas14 = "reviews-estados/review-Texas/14.json"
texas15 = "reviews-estados/review-Texas/15.json"
texas16 = "reviews-estados/review-Texas/16.json"

Creo una lista con los paths de cada json correspondiente a los tres estados:

In [105]:
paths=[[cal1, cal2 , cal3, cal4, cal5, cal6, cal7, cal8, cal9, cal10, cal11 , cal12, cal13, cal14, cal15, cal16, cal17, cal18],
       [florida1, florida2, florida3, florida4, florida5, florida6, florida7, florida8,florida9, florida10,florida11, florida12, florida13, florida14, florida15, florida16, florida17, florida18, florida19],
       [ texas1, texas2, texas3, texas4, texas5, texas6, texas7, texas8,texas9, texas10, texas11, texas12, texas13, texas14, texas15, texas16]]
       

Aplico la función para extraer, transformar y carga los datasets de California

In [106]:
gm_reviews= process_gm_reviews_files(paths)

Archivo CSV guardado con éxito


In [107]:
# Reviso: 
print(gm_reviews.info())
gm_reviews.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1908575 entries, 0 to 1908574
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   review_id  object 
 1   user_id    float64
 2   date       object 
 3   rating     int64  
 4   text       object 
 5   resp       object 
 6   gmap_id    object 
 7   state_iso  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 116.5+ MB
None


,review_id,user_id,date,rating,text,resp,gmap_id,state_iso
1908570,e6e7624f-30da-4599-92ac-1cb0e6ab14b2,1.001826e+20,2019-02-26,4,They have some of the best coffee and a kolach...,None,0x8640cb1bfe06768b:0x67ba7bb9c200582b,TX
1908571,2463258f-830c-4d2b-9b1d-739002e68576,1.165680e+20,2019-10-16,5,Great coffee and always fresh.,None,0x8640cb1bfe06768b:0x67ba7bb9c200582b,TX
1908572,af833767-0e7d-4667-a538-6e254ead91ab,1.029638e+20,2019-02-09,5,Opens early and very nice people who work there,None,0x8640cb1bfe06768b:0x67ba7bb9c200582b,TX
1908573,c4e5efbd-8c89-449a-833e-c87d0a46c2d4,1.008730e+20,2019-08-25,5,It's really close to work and as a new england...,None,0x8640cb1bfe06768b:0x67ba7bb9c200582b,TX
1908574,33d79681-b2b0-4165-92df-4b4a80e18ed3,1.093540e+20,2020-02-26,2,"Don't care for their Donuts, Shipley girl",None,0x8640cb1bfe06768b:0x67ba7bb9c200582b,TX


In [111]:
gm_reviews[gm_reviews['state_iso']=="TX"].shape

(469693, 8)

# YELP

### Archivo business.pkl

In [11]:
# Leo el archivo .pkl
yelp_metadata = pd.read_pickle('business.pkl')


In [12]:
# Reviso
print(yelp_metadata.info())
yelp_metadata.head()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


De lo anterior:
* Hay columns repetidas en las que casi el 100% son nulos. Conviene eliminarlas.
* La columna *is_open* es innecesaria ya que solo sería útil si brindara información en tiempo real.
* Reviso la columna *hours* para ver su contenido.

In [203]:
# Columna hours
yelp_metadata["hours"].head()

,hours,hours
0,None,NaN
1,"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",NaN
2,"{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",NaN
3,"{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NaN
4,"{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",NaN


Elimino las columnas repetidas:

In [204]:
yelp_metadata.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [205]:
# Elimino columnas duplicadas
yelp_metadata = yelp_metadata.loc[:, ~yelp_metadata.columns.duplicated()]
yelp_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


Elimino la columna *is_open*

In [206]:
yelp_metadata.drop("is_open", axis=1, inplace=True)


Hay registros en la columna postal_code que no pertenecen a EEUU sino a Canadá. Hay que ubicarlos:

In [207]:
pattern = r'^\d{5}$'  # Expresión regular para 5 dígitos
mask = yelp_metadata['postal_code'].str.match(pattern, na=False)
result = yelp_metadata[~mask]  # Obtener los valores que no coinciden
result

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
20,WKMJwqnfZKsAae75RMP6jA,Roast Coffeehouse and Wine Bar,10359 104 Street NW,Edmonton,LA,T5J 1B9,53.546045,-113.499169,4.0,40,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re...","{'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ..."
46,JX4tUpd09YFchLBuI43lGw,Naked Cyber Cafe & Espresso Bar,10303 108 Street NW,Edmonton,PA,T5J 1L7,53.544682,-113.506589,4.0,12,"{'OutdoorSeating': 'False', 'BusinessParking':...","Arts & Entertainment, Music Venues, Internet S...","{'Monday': '11:0-1:0', 'Tuesday': '11:0-1:0', ..."
52,5BmQX4UVJY19mMtafMg7JA,Breadland Organic Whole Grain Bakery,"11642 104 Avenue, Unit 642",Edmonton,PA,T5K 2T7,53.547205,-113.522448,4.0,23,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Specialty Food, Bakeries, Food, Health Markets","{'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ..."
53,cVBxfMC4lp3DnocjYA3FHQ,Paws The Cat Cafe,10588 109 Street,Edmonton,FL,T5H 3B2,53.549633,-113.50878,5.0,20,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Coffee & Tea, Cafes, Pets, Restaurants, Pet Ad...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-21:0', ..."
77,mFTaYsRDyeFaY2NDLfSaAA,River City Games,5203 99 Street NW,Edmonton,FL,T6E 5B7,53.489312,-113.485567,4.5,5,"{'BusinessParking': '{'garage': False, 'street...","Tabletop Games, Shopping, Sporting Goods","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150263,VRSQk67FPiDQolDyiaFnjg,Citytv,10212 Jasper Ave NW,Edmonton,PA,T5J 5A3,53.541168,-113.495953,4.0,8,None,"Television Stations, Mass Media",None
150276,Rn3DtOiuJtc3whFySWXs5Q,Japanese Village,2940 Calgary Trail NW,Edmonton,PA,T6J 7C2,53.463917,-113.494702,3.0,35,"{'RestaurantsAttire': 'u'casual'', 'NoiseLevel...","Japanese, Restaurants, Steakhouses, Sushi Bars","{'Monday': '0:0-0:0', 'Tuesday': '17:0-22:0', ..."
150332,t_SGoRT5yt14OWr64TOulA,Sherwood Park Kwik Lube,979 Fir St,Sherwood Park,PA,T8A 4N5,53.513215,-113.32868,5.0,5,None,"Oil Change Stations, Automotive, Auto Repair",None
150340,hn9Toz3s-Ei3uZPt7esExA,West Side Kebab House,2470 Guardian Road NW,Edmonton,NJ,T5T 1K8,53.509649,-113.675999,4.5,18,"{'Ambience': '{'touristy': False, 'hipster': F...","Middle Eastern, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'..."


Deben eliminarse:

In [208]:
yelp_metadata = yelp_metadata[mask]
yelp_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144716 entries, 0 to 150345
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   144716 non-null  object
 1   name          144716 non-null  object
 2   address       144716 non-null  object
 3   city          144716 non-null  object
 4   state         144713 non-null  object
 5   postal_code   144716 non-null  object
 6   latitude      144716 non-null  object
 7   longitude     144716 non-null  object
 8   stars         144716 non-null  object
 9   review_count  144716 non-null  object
 10  attributes    131587 non-null  object
 11  categories    144614 non-null  object
 12  hours         122557 non-null  object
dtypes: object(13)
memory usage: 15.5+ MB


Al parecer, hay ciudades que no corresponden al estado que tienen. Los estados de interés con respecto al objetivo son California, Florida y Texas. A partir de los rangos de código postal provistos en la página [codigo-postal](https://codigo-postal.co/en-us/usa/) obtengo los estados correctos:

In [209]:
# Primero, imputo todos los estados con "-", para después obtener solo los que nos interesan:
yelp_metadata['state']='-'

# CALIFORNIA
mask_ca = (yelp_metadata['postal_code'].astype(int) >= 90001) & (yelp_metadata['postal_code'].astype(int) <= 96162)
yelp_metadata.loc[mask_ca, 'state'] = 'CA'

# FLORIDA           
mask_fl = (yelp_metadata['postal_code'].astype(int) >= 32003) & (yelp_metadata['postal_code'].astype(int) <= 34997)
yelp_metadata.loc[mask_fl, 'state'] = 'FL'

# TEXAS
mask_tx = (yelp_metadata['postal_code'].astype(int) >= 73301) & (yelp_metadata['postal_code'].astype(int) <= 85000)
yelp_metadata.loc[mask_tx, 'state'] = 'TX'  

# Reviso:
yelp_metadata.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,-,63123,38.551126,-90.335695,3.0,15,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,-,85711,32.223236,-110.880452,3.5,22,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,-,19107,39.955505,-75.155564,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,-,18054,40.338183,-75.471659,4.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,-,37015,36.269593,-87.058943,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '..."
6,n_0UpQx1hsNbnPUSlodU8w,Famous Footwear,"8522 Eager Road, Dierbergs Brentwood Point",Brentwood,-,63144,38.627695,-90.340465,2.5,13,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ..."
7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.76659,-82.732983,3.5,5,None,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,-,63123,38.565165,-90.321087,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,-,37207,36.208102,-86.76817,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '..."


In [210]:
# Filtro por CA, FL, TX
yelp_metadata= yelp_metadata[yelp_metadata['state'].isin(["CA", "FL", "TX"])]
yelp_metadata.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
7,qkRM_2X51Yqxk3btlwAQIg,Temple Beth-El,400 Pasadena Ave S,St. Petersburg,FL,33707,27.76659,-82.732983,3.5,5,None,"Synagogues, Religious Organizations","{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ..."


Reviso cómo está compuesta la columna "categories"

In [211]:
# Reviso el tipo de dato: contiene strings
type(yelp_metadata['categories'][0])

str

In [212]:
# Reviso el contenido
yelp_metadata['categories'].head(20)

0     Doctors, Traditional Chinese Medicine, Naturop...
7                   Synagogues, Religious Organizations
10                 Department Stores, Shopping, Fashion
11           Vietnamese, Food, Restaurants, Food Trucks
13    General Dentistry, Dentists, Health & Medical,...
14          Food, Delis, Italian, Bakeries, Restaurants
26    Women's Clothing, Accessories, Children's Clot...
30                        Trampoline Parks, Active Life
50    Active Life, Fitness & Instruction, Trainers, ...
54                    Food, Grocery, Convenience Stores
58                 Restaurants, American (New), Italian
59                                   Restaurants, Pizza
63                                   Uniforms, Shopping
66                  Local Services, Appliances & Repair
69                                        Food, Grocery
74    Health & Medical, Personal Care Services, Beau...
79    Burgers, Sports Bars, Bars, Lounges, Restauran...
85    Food, Restaurants, Salad, Coffee & Tea, Br

Filtro por restaurantes

In [213]:
# Filtro las filas que contienen la palabra "Restaurants" en la columna 'categories'
yelp_metadata = yelp_metadata[yelp_metadata['categories'].apply(lambda x: x is not None and 'Restaurants' in x)]

# Convierto cada registro en una list
yelp_metadata["categories"] = yelp_metadata["categories"].str.split(",")



In [214]:
# Reviso el nuevo tipo de dato
type(yelp_metadata["categories"][11])

list

In [215]:
# Reviso:
print(yelp_metadata.info())
yelp_metadata.categories.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 11195 entries, 11 to 150327
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   11195 non-null  object
 1   name          11195 non-null  object
 2   address       11195 non-null  object
 3   city          11195 non-null  object
 4   state         11195 non-null  object
 5   postal_code   11195 non-null  object
 6   latitude      11195 non-null  object
 7   longitude     11195 non-null  object
 8   stars         11195 non-null  object
 9   review_count  11195 non-null  object
 10  attributes    11071 non-null  object
 11  categories    11195 non-null  object
 12  hours         9872 non-null   object
dtypes: object(13)
memory usage: 1.4+ MB
None


11      [Vietnamese,  Food,  Restaurants,  Food Trucks]
14    [Food,  Delis,  Italian,  Bakeries,  Restaurants]
58             [Restaurants,  American (New),  Italian]
Name: categories, dtype: object

Reviso duplicados en *business_id*:

In [216]:
yelp_metadata.business_id.duplicated().sum()


0

Elimino la columna *hours*:

In [217]:
yelp_metadata.drop("hours", axis=1, inplace=True)
yelp_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11195 entries, 11 to 150327
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   11195 non-null  object
 1   name          11195 non-null  object
 2   address       11195 non-null  object
 3   city          11195 non-null  object
 4   state         11195 non-null  object
 5   postal_code   11195 non-null  object
 6   latitude      11195 non-null  object
 7   longitude     11195 non-null  object
 8   stars         11195 non-null  object
 9   review_count  11195 non-null  object
 10  attributes    11071 non-null  object
 11  categories    11195 non-null  object
dtypes: object(12)
memory usage: 1.4+ MB


De lo anterior, se observa que hay solo 117 nulos en la columna *attributes*. Ya que son pocos y los datos de esa columna son importantes para nuestros posteriores modelos de ml, los imputamos con "No data":

In [218]:
yelp_metadata.fillna("No data", inplace=True)
yelp_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11195 entries, 11 to 150327
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   11195 non-null  object 
 1   name          11195 non-null  object 
 2   address       11195 non-null  object 
 3   city          11195 non-null  object 
 4   state         11195 non-null  object 
 5   postal_code   11195 non-null  object 
 6   latitude      11195 non-null  float64
 7   longitude     11195 non-null  float64
 8   stars         11195 non-null  float64
 9   review_count  11195 non-null  int64  
 10  attributes    11195 non-null  object 
 11  categories    11195 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 1.4+ MB


Revisamos el postal_code:

In [219]:
yelp_metadata.postal_code.head(5)

11    33602
14    33771
58    33618
59    33607
79    33544
Name: postal_code, dtype: object

Reindexo:

In [220]:
# Reiniciar los índices 
yelp_metadata.reset_index(drop=True, inplace=True)

# Reviso:
yelp_metadata.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","[Vietnamese, Food, Restaurants, Food Trucks]"
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","[Food, Delis, Italian, Bakeries, Restaurants]"


Ya el dataframe está limpio y transformado de acuerdo a nuestros objetivos. 
Finalmente, se divide en tablas para el MER:

Normalizo primero la columna city:

In [221]:
yelp_metadata['city']= yelp_metadata['city'].str.title()
yelp_metadata.city.tail(5)

11190     Santa Barbara
11191             Boise
11192    St. Petersburg
11193             Boise
11194             Boise
Name: city, dtype: object

Tabla city:

In [222]:
# Creo un nuevo df con los valores únicos de  'postal_code' y seleccionor las columnas 'postal_code' y 'city'
cities = yelp_metadata.drop_duplicates(subset=['city'])[['city','state']]
cities= cities.sort_values(by=['state', 'city'])

# Reindexo
cities.reset_index(drop=True, inplace=True)

# Generar un identificador único para city-postal_code:
cities['city_id'] = cities['city'].str[:2].str.lower() + "-" + (pd.Series(range(100, 100+cities.shape[0]))).astype(str)
 
# Reordeno columnas
cities= cities[['city_id', 'city', 'state']]

# Reviso:
cities

,city_id,city,state
0,ca-100,Carpinteria,CA
1,go-101,Goleta,CA
2,is-102,Isla Vista,CA
3,lu-103,Lula Lula,CA
4,mo-104,Montecito,CA
...,...,...,...
123,ku-223,Kuna,TX
124,me-224,Meridian,TX
125,me-225,Meridian,TX
126,na-226,Nampa,TX


Agrego la columna city_id a yelp_metadata:

In [223]:
# Crear un diccionario de mapeo para la columna 'city_id'
city_id_mapping = cities.set_index('city')['city_id']

# Usar el método .map() para asignar los valores de 'city_id' en 'yelp_metadata'
yelp_metadata['city_id'] = yelp_metadata['city'].map(city_id_mapping)

# Revisar
yelp_metadata


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,city_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","[Vietnamese, Food, Restaurants, Food Trucks]",ta-195
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,Largo,FL,33771,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","[Food, Delis, Italian, Bakeries, Restaurants]",la-139
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,Tampa,FL,33618,28.046203,-82.505053,4.0,23,"{'BusinessParking': '{'garage': False, 'street...","[Restaurants, American (New), Italian]",ta-194
3,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,Tampa,FL,33607,27.960514,-82.506127,4.0,35,"{'BusinessParking': '{'garage': False, 'street...","[Restaurants, Pizza]",ta-194
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,Wesley Chapel,FL,33544,28.196252,-82.380615,4.5,95,"{'BestNights': '{'monday': False, 'tuesday': F...","[Burgers, Sports Bars, Bars, Lounges, Rest...",we-213
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11190,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,Santa Barbara,CA,93105,34.440689,-119.739681,4.0,299,"{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...","[Restaurants, Pizza, Food]",sa-106
11191,-R9t30RvsG-Rtiq63yuEmg,Crescent No Lawyers Bar-Grill,5500 W Franklin Rd,Boise,TX,83706,43.603368,-116.250063,2.5,55,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","[American (Traditional), Nightlife, Bars, R...",bo-218
11192,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,St. Petersburg,FL,33701,27.770872,-82.643069,4.5,38,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","[Bars, Beer Bar, Nightlife, Wine Bars, Piz...",st-190
11193,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,Boise,TX,83702,43.616590,-116.202383,4.5,998,"{'BikeParking': 'True', 'Alcohol': 'u'full_bar...","[Bars, Gastropubs, Sandwiches, Nightlife, ...",bo-218


Elimino las columnas *city*, *postal_code* y *state*:

In [224]:
yelp_metadata.drop(['city', 'postal_code', 'state'], axis=1, inplace=True)


In [225]:
yelp_metadata.columns.values

array(['business_id', 'name', 'address', 'latitude', 'longitude', 'stars',
       'review_count', 'attributes', 'categories', 'city_id'],
      dtype=object)

Agrego la columna *state_id* en el df cities:

In [228]:
states= pd.read_csv('POST_ETL_DATASETS\Google_Maps_Data\states.csv')
states

,state_id,iso,name
0,1,CA,California
1,2,FL,Florida
2,3,TX,Texas


In [230]:
# Crear un diccionario de mapeo para la columna 'state_id'
state_id_mapping = states.set_index('iso')['state_id']

# Usar el método .map() para asignar los valores de 'state_id' en 'cities'
cities['state_id'] = cities['state'].map(state_id_mapping)

# Revisar
cities


,city_id,city,state,state_id
0,ca-100,Carpinteria,CA,1
1,go-101,Goleta,CA,1
2,is-102,Isla Vista,CA,1
3,lu-103,Lula Lula,CA,1
4,mo-104,Montecito,CA,1
...,...,...,...,...
123,ku-223,Kuna,TX,3
124,me-224,Meridian,TX,3
125,me-225,Meridian,TX,3
126,na-226,Nampa,TX,3


Por último, elimino la columna state:

In [231]:
cities.drop('state', axis=1, inplace=True)
cities.head(2)

,city_id,city,state_id
0,ca-100,Carpinteria,1
1,go-101,Goleta,1


In [232]:
cities

,city_id,city,state_id
0,ca-100,Carpinteria,1
1,go-101,Goleta,1
2,is-102,Isla Vista,1
3,lu-103,Lula Lula,1
4,mo-104,Montecito,1
...,...,...,...
123,ku-223,Kuna,3
124,me-224,Meridian,3
125,me-225,Meridian,3
126,na-226,Nampa,3


In [233]:
yelp_metadata

,business_id,name,address,latitude,longitude,stars,review_count,attributes,categories,city_id
0,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,27.955269,-82.456320,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","[Vietnamese, Food, Restaurants, Food Trucks]",ta-195
1,0bPLkL0QhhPO5kt1_EXmNQ,Zio's Italian Market,2575 E Bay Dr,27.916116,-82.760461,4.5,100,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","[Food, Delis, Italian, Bakeries, Restaurants]",la-139
2,uI9XODGY_2_ieTE6xJ0myw,Roman Forum,10440 N Dale Mabry Hwy,28.046203,-82.505053,4.0,23,"{'BusinessParking': '{'garage': False, 'street...","[Restaurants, American (New), Italian]",ta-194
3,JgpnXv_0XhV3SfbfB50nxw,Joe's Pizza,2038 N Dale Mabry Hwy,27.960514,-82.506127,4.0,35,"{'BusinessParking': '{'garage': False, 'street...","[Restaurants, Pizza]",ta-194
4,pJfh3Ct8iL58NZa8ta-a5w,Top Shelf Sports Lounge,3173 Cypress Ridge Blvd,28.196252,-82.380615,4.5,95,"{'BestNights': '{'monday': False, 'tuesday': F...","[Burgers, Sports Bars, Bars, Lounges, Rest...",we-213
...,...,...,...,...,...,...,...,...,...,...
11190,uriD7RFuHhLJeDdKaf0nFA,Pizza Guru,3534 State St,34.440689,-119.739681,4.0,299,"{'NoiseLevel': 'u'average'', 'RestaurantsGoodF...","[Restaurants, Pizza, Food]",sa-106
11191,-R9t30RvsG-Rtiq63yuEmg,Crescent No Lawyers Bar-Grill,5500 W Franklin Rd,43.603368,-116.250063,2.5,55,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","[American (Traditional), Nightlife, Bars, R...",bo-218
11192,esBGrrmuZzSiECyRBoKvvA,Colony Grill - St. Petersburg,670 Central Ave,27.770872,-82.643069,4.5,38,"{'RestaurantsPriceRange2': '2', 'RestaurantsAt...","[Bars, Beer Bar, Nightlife, Wine Bars, Piz...",st-190
11193,w_4xUt-1AyY2ZwKtnjW0Xg,Bittercreek Alehouse,246 N 8th St,43.616590,-116.202383,4.5,998,"{'BikeParking': 'True', 'Alcohol': 'u'full_bar...","[Bars, Gastropubs, Sandwiches, Nightlife, ...",bo-218


### FUNCIÓN process_yelp_metadata:

Ya todas las transformaciones están realizadas. Ahora se integran todas en una función:

In [235]:
def process_yelp_metadata(path):
# EXTRACCIÓN   
    # Leo el archivo .pkl
    yelp_metadata = pd.read_pickle(path)

# TRANSFORMACIÓN    
    # Elimino columnas duplicadas
    yelp_metadata = yelp_metadata.loc[:, ~yelp_metadata.columns.duplicated()]

    # Elimino columnas innecesarias
    yelp_metadata.drop(["is_open", "hours"], axis=1, inplace=True)

    # Hay registros en la columna postal_code que no pertenecen a EEUU sino a Canadá. Hay que eliminarlas:
    pattern = r'^\d{5}$'  # Expresión regular para 5 dígitos
    mask = yelp_metadata['postal_code'].str.match(pattern, na=False)
    yelp_metadata = yelp_metadata[mask] # Filtro los registros 

    # Asigno los estados correctos a las ciudades
    # Primero imputo todos los estados con "-", para después obtener solo los que nos interesan:
    yelp_metadata['state']='-'

    # CALIFORNIA
    mask_ca = (yelp_metadata['postal_code'].astype(int) >= 90001) & (yelp_metadata['postal_code'].astype(int) <= 96162)
    yelp_metadata.loc[mask_ca, 'state'] = 'CA'

    # FLORIDA           
    mask_fl = (yelp_metadata['postal_code'].astype(int) >= 32003) & (yelp_metadata['postal_code'].astype(int) <= 34997)
    yelp_metadata.loc[mask_fl, 'state'] = 'FL'

    # TEXAS
    mask_tx = (yelp_metadata['postal_code'].astype(int) >= 73301) & (yelp_metadata['postal_code'].astype(int) <= 85000)
    yelp_metadata.loc[mask_tx, 'state'] = 'TX'

    # Filtro por CA, FL, TX
    yelp_metadata= yelp_metadata[yelp_metadata['state'].isin(["CA", "FL", "TX"])]

    # Filtro las filas que contienen la palabra "Restaurants" en la columna 'categories'
    yelp_metadata = yelp_metadata[yelp_metadata['categories'].apply(lambda x: x is not None and 'Restaurants' in x)]

    # Convierto cada registro en una list
    yelp_metadata["categories"] = yelp_metadata["categories"].str.split(",")

    # Imputo nulos :
    yelp_metadata.fillna("No data", inplace=True)

    # Reiniciar los índices 
    yelp_metadata.reset_index(drop=True, inplace=True)

    # Normalizo la columna "city"
    yelp_metadata['city']= yelp_metadata['city'].str.title()

    # TABLA "cities"

    # Creo un nuevo df con los valores únicos de  'postal_code' y seleccionor las columnas 'postal_code' y 'city'
    cities = yelp_metadata.drop_duplicates(subset=['city'])[['city','state']]
    cities= cities.sort_values(by=['state', 'city'])

    # Reindexo
    cities.reset_index(drop=True, inplace=True)

    # Generar un identificador único para city-postal_code:
    cities['city_id'] = cities['city'].str[:2].str.lower() + "-" + (pd.Series(range(100, 100+cities.shape[0]))).astype(str)
    
    # Reordeno columnas
    cities= cities[['city_id', 'city', 'state']]

    #Agrego la columna "city_id" al df
    # Crear un diccionario de mapeo para la columna 'city_id'
    city_id_mapping = cities.set_index('city')['city_id']

    # Usar el método .map() para asignar los valores de 'city_id' en 'yelp_metadata'
    yelp_metadata['city_id'] = yelp_metadata['city'].map(city_id_mapping)

    # Elimino las columnas repetitivas:
    yelp_metadata.drop(['city', 'postal_code', 'state'], axis=1, inplace=True)

    # Obtengo el csv "states" para crear la columna "state_id"
    states= pd.read_csv('POST_ETL_DATASETS/Google_Maps_Data/states.csv')

    # Crear un diccionario de mapeo para la columna 'state_id'
    state_id_mapping = states.set_index('iso')['state_id']

    # Usar el método .map() para asignar los valores de 'state_id' en 'cities'
    cities['state_id'] = cities['state'].map(state_id_mapping)

    # Eliminar la columna "state"
    cities.drop('state', axis=1, inplace=True)
    
# CARGA
    yelp_metadata.to_csv('POST_ETL_DATASETS/Yelp_Data/yelp_restaurants.csv', index=False, encoding="utf-8")
    cities.to_csv('POST_ETL_DATASETS/Yelp_Data/cities.csv', index=False, encoding="utf-8")
    print(f"Archivos csv guardados con éxito.")
    
    return yelp_metadata, cities
   

In [236]:
yelp_restaurants, cities_df= process_yelp_metadata("business.pkl")

Archivos csv guardados con éxito.


In [237]:
yelp_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11195 entries, 0 to 11194
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   11195 non-null  object 
 1   name          11195 non-null  object 
 2   address       11195 non-null  object 
 3   latitude      11195 non-null  float64
 4   longitude     11195 non-null  float64
 5   stars         11195 non-null  float64
 6   review_count  11195 non-null  int64  
 7   attributes    11195 non-null  object 
 8   categories    11195 non-null  object 
 9   city_id       11195 non-null  object 
dtypes: float64(3), int64(1), object(6)
memory usage: 874.7+ KB


In [239]:
cities_df.head()

,city_id,city,state_id
0,ca-100,Carpinteria,1
1,go-101,Goleta,1
2,is-102,Isla Vista,1
3,lu-103,Lula Lula,1
4,mo-104,Montecito,1


### Archivo review.json

El archivo es demasiado grande, por lo que extraigo las primeras 1000000 filas

In [3]:
# Obtengo el archivo json en un dataframe:
yelp_reviews= pd.read_json('C:/Users/Usuario/Downloads/review.json', lines=True, nrows=1000000)


Reviso la información del df

In [4]:
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   review_id    1000000 non-null  object        
 1   user_id      1000000 non-null  object        
 2   business_id  1000000 non-null  object        
 3   stars        1000000 non-null  int64         
 4   useful       1000000 non-null  int64         
 5   funny        1000000 non-null  int64         
 6   cool         1000000 non-null  int64         
 7   text         1000000 non-null  object        
 8   date         1000000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 68.7+ MB


In [8]:
yelp_reviews.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


Reviso variables numéricas:

In [5]:
yelp_reviews.describe()

,stars,useful,funny,cool,date
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000
mean,3.783959,1.106594,0.307812,0.457129,2016-07-13 10:09:02.464016128
min,1.000000,0.000000,0.000000,0.000000,2005-03-01 17:47:15
25%,3.000000,0.000000,0.000000,0.000000,2014-08-07 16:29:16.500000
50%,4.000000,0.000000,0.000000,0.000000,2016-10-11 16:02:42
75%,5.000000,1.000000,0.000000,0.000000,2018-09-01 01:26:22.249999872
max,5.000000,539.000000,284.000000,172.000000,2022-01-19 19:47:59
std,1.440197,2.848906,1.428185,1.799114,NaN


De lo anterior:
* La columna *date* debe ser cambiada a formato fecha, ya que está como numérica.
* Elimino columnas innecesarias *useful*, *funny* y *cool*:

In [10]:
yelp_reviews.drop(['useful', 'funny', 'cool'], axis=1, inplace=True)
yelp_reviews.head(2)

,review_id,user_id,business_id,stars,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18


Me quedo solo con la fechea de *date*:

In [12]:
# Extraigo solo la fecha 
yelp_reviews['date'] = yelp_reviews['date'].dt.date

yelp_reviews.head(2)


,review_id,user_id,business_id,stars,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,"If you decide to eat here, just be aware it is...",2018-07-07
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,I've taken a lot of spin classes over the year...,2012-01-03


Filtro por los códigos de los restaurantes de CA, FL y TX, utilizando el df anterior yelp_metadata

In [17]:
# Genero una lista de los gmap_id de los restaurantes del df gmap_metadata:
lista_id_restaurantes= list(yelp_metadata['business_id'])

# Reviso
lista_id_restaurantes

['MTSW4McQd7CbVtyjqoe9mw',
 '0bPLkL0QhhPO5kt1_EXmNQ',
 'sqSqqLy0sN8n2IZrAbzidQ',
 'cVBxfMC4lp3DnocjYA3FHQ',
 'seKihQKpGGnCeLuELRQPSQ',
 'L_TT0BFmFwORAMaA83K54A',
 'ppFCk9aQkM338Rgwpl2F5A',
 'LhpPSrulqVeTyJeK2xydvQ',
 'FRYkg_JvsWU9xIXZsEZcVA',
 '7Du9oW73YcYFmXdtU5aiSg',
 'QjV4v7q_pt7tt3K1US7IHg',
 'LGqiubTmpJ-A1L5n7dmc6g',
 '59jkCUm4yHciKL1KW_1HJA',
 'vPcfJ3rm3NpdqVDod2WxLA',
 '8c0r7olQSYGcws0bTd3ikw',
 'uDQgmudYDKiB6n4vwbEeDg',
 'Edg22x3CZkIv0GUib2oEFA',
 'n8ecak12IF_jhnPNs37AZA',
 'jcL_qaGJiappzpnn-ifSoA',
 'W57cR9a7XP6RX56MSWHJJA',
 'xa6JYHDgVza7CuenKBJBHw',
 '-7GDqSUaXrpC8Ql7nDBxWA',
 'XwoXi0M66RU6Gouq-DSDHw',
 'OfW-VK04qr7b7bPm5QbyLg',
 'JfGfqCnk0-Tnf6PlahPUJQ',
 'GVqXNt0HKJLpciN9ePxnWw',
 'iBRXLFWD7PeSYDbKsd_JTw',
 'od6skmfXz9twktEAuJHEmw',
 'sUQ-OfJYSNfz-cimEjoaXQ',
 'z9wCTHYI2VZy9YIblwSsgg',
 'QWqKTWQ2OiDgo3dzNkpung',
 'fNssGWMjae-SUzyyGblspQ',
 'bPPy7yNFfb-6N_bHkn1qAw',
 'D7z_V6E8cxXghSRcHAmUPg',
 'JPm93BlP-UHYPqFgK66JUw',
 'pJB63RaQpkTIKBhfTsLQQg',
 'SlLfWzeYOrVBxCd-0QILZQ',
 

In [18]:
# Filtro los establecimientos que son restaurantes y están en CA, FL y TX a partir del business_id de la lista anterior:
yelp_reviews = yelp_reviews[yelp_reviews['business_id'].isin(lista_id_restaurantes)]

# Reviso:
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144155 entries, 7 to 999980
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    144155 non-null  object
 1   user_id      144155 non-null  object
 2   business_id  144155 non-null  object
 3   stars        144155 non-null  int64 
 4   text         144155 non-null  object
 5   date         144155 non-null  object
dtypes: int64(1), object(5)
memory usage: 7.7+ MB


In [28]:
yelp_reviews.head()

,review_id,user_id,business_id,stars,text,date
0,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,Amazingly amazing wings and homemade bleu chee...,2015-08-07
1,pUycOfUwM8vqX7KjRRhUEA,59MxRhNVhU9MYndMkz0wtw,gebiRewfieSdtt17PTW6Zg,3,Had a party of 6 here for hibachi. Our waitres...,2016-07-25
2,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4,The bun makes the Sonoran Dog. It's like a snu...,2011-10-27
3,z0osLHDvXvzfm57D4DmD2Q,xVKE_HJ2pwUtTdLbL3pnCg,S2Ho8yLxhKAa26pBAm6rxA,3,"Service was crappy, and food was mediocre. I ...",2016-11-22
4,-P5E9BYUaK7s3PwBF5oAyg,Jha0USGDMefGFRLik_xFQg,bMratNjTG5ZFEA6hVyr-xQ,5,First time there and it was excellent!!! It fe...,2017-02-19


Filtro a partir del año 2018

In [32]:
yelp_reviews=yelp_reviews[yelp_reviews['date'].apply(lambda x: x.year)>=2018]
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 48716 entries, 9 to 144148
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    48716 non-null  object
 1   user_id      48716 non-null  object
 2   business_id  48716 non-null  object
 3   stars        48716 non-null  int64 
 4   text         48716 non-null  object
 5   date         48716 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.6+ MB


In [35]:
# Reviso la fecha máxima:
yelp_reviews.date.max()

datetime.date(2022, 1, 19)

Reviso duplicados

In [30]:
yelp_reviews.duplicated().sum()

0

In [31]:
yelp_reviews.drop_duplicates()

,review_id,user_id,business_id,stars,text,date
9,4KpIldEM-tdnrJLqYzRfZQ,Z5j9Xw_G0c7M2b1-iS67wg,HTqXI5S2XcSlh_ylx9sE6g,5,I've only had the cannolis here but they are a...,2018-03-23
12,tXHWJWnTdrraHGUqaPWj3g,zKAHSNzqvwsyoFCw3QpafA,S2Ho8yLxhKAa26pBAm6rxA,4,Enjoyed my fish out at a sidewalk table. A bi...,2018-05-19
33,NyJwwd6Ylajd9sVAaIZ9Iw,s2Bm2i1w24_xgBhPw9xzlA,gZqFuqTjtN4Bfh-_fO315w,5,Very impressed with this place! The food was a...,2018-06-19
54,LUsxbDfNYFTrX6y_Ist3Pw,DtxpEKBjSfTFPHCf_YOYCA,eJYvM2Zy5BloOYGldWmAVw,5,OMG the Mediterranean chicken quesadillas what...,2018-08-14
85,xhhmoxmvIJp16ozmxBuZZg,iiRTJ3x2Z4S3d5ZZ24vmdA,yE1raqkLX7OZsjmX3qKIKg,2,We arrived for lunch at 12p on a Friday - wasn...,2018-04-27
...,...,...,...,...,...,...
144137,13ZzF3ET_pcLkaFhvBc2FA,7RsxDnE_bp9Hd1rDMzZXfA,UFCN0bYdHroPKu6KV5CJqg,5,I've come to the realization that the chicken ...,2018-01-06
144139,s78yT8E3m64B0-QD17QLuw,_lC-l5WnF2lJ9h7-SXH5Vg,hXvf5a798WvQtFn0Zbh2yw,5,Never have been disappointed coming here. Grea...,2019-01-28
144142,RAeqvrXhuT9hh30riFzPzg,UUJLEaupUPDG7DDRXcSfEw,NHb6QTrJOnyIj-Zd0fYzFg,5,"This was my first visit there, so craving some...",2018-03-02
144147,PG8JgbhpMswU1rZ5tA3ZqQ,v2_9bXToNtdcmfeoVKZ0eA,MXTV7gLn1GcgBIskfIXwMg,5,This is without a doubt the number one Restaur...,2018-05-26


Reindexo 

In [33]:
yelp_reviews.reset_index(drop=True, inplace=True)
yelp_reviews.head(2)

,review_id,user_id,business_id,stars,text,date
0,4KpIldEM-tdnrJLqYzRfZQ,Z5j9Xw_G0c7M2b1-iS67wg,HTqXI5S2XcSlh_ylx9sE6g,5,I've only had the cannolis here but they are a...,2018-03-23
1,tXHWJWnTdrraHGUqaPWj3g,zKAHSNzqvwsyoFCw3QpafA,S2Ho8yLxhKAa26pBAm6rxA,4,Enjoyed my fish out at a sidewalk table. A bi...,2018-05-19


### 4. FUNCIÓN process_yelp_reviews_file():

Por último, creo una función que abra el json por tandas y le aplique a cada una las transformaciones anteriores, y al final devuelve un dataframe unido:

In [241]:
def process_yelp_reviews_file(path, chunk_size, df_metadata=None):
    # Extraigo el df a partir del cual se realizará un filtro luego:
    if df_metadata is None:
        df_metadata = pd.read_csv("POST_ETL_DATASETS/Yelp_Data/yelp_restaurants.csv")
    
    
    # Crear una lista para almacenar los DataFrames procesados de cada trozo
    chunks_procesados = []
    
    # Leer el archivo JSON en trozos y procesarlos por separado
    for chunk in pd.read_json(path, lines=True, chunksize=chunk_size):
        # Eliminar columnas innecesarias
        chunk.drop(['useful', 'funny', 'cool'], axis=1, inplace=True)

        # Extraer solo la fecha de la columna "date"
        chunk['date'] = pd.to_datetime(chunk['date']).dt.date
            
        # Generar lista de business_id de restaurantes del yelp_restaurants
        lista_id_restaurantes = list(df_metadata['business_id'])

        # Filtrar establecimientos que son restaurantes y están en CA, FL y TX
        chunk = chunk[chunk['business_id'].isin(lista_id_restaurantes)]
            
        # Filtrar a partir del año 2018
        chunk=chunk[chunk['date'].apply(lambda x: x.year)>=2018]
        
        # Agregar el trozo procesado a la lista de trozos procesados
        chunks_procesados.append(chunk)
     
    # Concatenar los chunks de la lista en un solo dataframe    
    df_concatenado= pd.concat(chunks_procesados)

    # Eliminar duplicados y reindexar
    df_concatenado.drop_duplicates(inplace=True)
    df_concatenado.reset_index(drop=True, inplace=True)

# CARGA
    df_concatenado.to_csv('POST_ETL_DATASETS/Yelp_Data/yelp_reviews.csv', index=False, encoding="utf-8")
    print("Archivo guardado con éxito.")
    
    return df_concatenado



In [242]:
# Defino el path:
path = 'C:/Users/Usuario/Downloads/review.json'
chunk_size = 100000  # Aquí puedes ajustar el tamaño de la tanda según tus necesidades

In [243]:
yelp_reviews = process_yelp_reviews_file(path, chunk_size)

Archivo guardado con éxito.


In [244]:
# Reviso
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525475 entries, 0 to 525474
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   review_id    525475 non-null  object
 1   user_id      525475 non-null  object
 2   business_id  525475 non-null  object
 3   stars        525475 non-null  int64 
 4   text         525475 non-null  object
 5   date         525475 non-null  object
dtypes: int64(1), object(5)
memory usage: 24.1+ MB


In [245]:
yelp_reviews.date.max()

datetime.date(2022, 1, 19)

### Archivo user.parquet

Extraigo el archivo parquet:

In [ ]:
yelp_users=pd.read_parquet("user.parquet")

In [39]:
# Reviso:
print(yelp_users.info())
yelp_users.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB
None


,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


De lo anterior:
* Eliminar columnas inncesarias: *yelping_since*, *useful*, *funny*, *cool* y todos los *compliments* excepto *compliment_photos*.
* De la columna *fans* se puede para calcular la influencia. 
* No se ven nulos. Se debe filtrar el df solo con los usuarios que hicieron reviews en los restaurantes de CA, FL y TX.


Elimino columnas innecesarias:

In [42]:
# Obtengo todas las columnas:
yelp_users.columns.values

array(['user_id', 'name', 'review_count', 'yelping_since', 'useful',
       'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars',
       'compliment_hot', 'compliment_more', 'compliment_profile',
       'compliment_cute', 'compliment_list', 'compliment_note',
       'compliment_plain', 'compliment_cool', 'compliment_funny',
       'compliment_writer', 'compliment_photos'], dtype=object)

In [43]:
# Hago una lista con las columnas a eliminar:
columnas= ['yelping_since', 'useful','funny', 'cool', 'friends'
           'compliment_hot', 'compliment_more', 'compliment_profile',
           'compliment_cute', 'compliment_list', 'compliment_note',
           'compliment_plain', 'compliment_cool', 'compliment_funny',
           'compliment_writer']


In [45]:
# Elimino las columnas:
yelp_users.drop(columnas, axis=1, inplace =True)

# Reviso:
yelp_users.columns.values

array(['user_id', 'name', 'review_count', 'elite', 'friends', 'fans',
       'average_stars', 'compliment_photos'], dtype=object)

In [46]:
# Traigo el dataset de yelp_reviews:
yelp_reviews= pd.read_csv('POST_ETL_DATASETS\yelp_reviews.csv')
yelp_reviews.head(2)

,review_id,user_id,business_id,stars,text,date
0,4KpIldEM-tdnrJLqYzRfZQ,Z5j9Xw_G0c7M2b1-iS67wg,HTqXI5S2XcSlh_ylx9sE6g,5,I've only had the cannolis here but they are a...,2018-03-23
1,tXHWJWnTdrraHGUqaPWj3g,zKAHSNzqvwsyoFCw3QpafA,S2Ho8yLxhKAa26pBAm6rxA,4,Enjoyed my fish out at a sidewalk table. A bi...,2018-05-19


Obtengo en una lista los user_id únicos de yelp_reviews:

In [ ]:
lista_user_id= list(yelp_reviews['user_id'].unique())

In [50]:
# Reviso:
len(lista_user_id)

258719

Filtro los usuarios a partir de la lista generada:

In [51]:
yelp_users = yelp_users[yelp_users['user_id'].isin(lista_user_id)]

In [55]:
# Reviso:
print(yelp_users.info())
yelp_users.head()

<class 'pandas.core.frame.DataFrame'>
Index: 277248 entries, 1 to 2105596
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            277248 non-null  object 
 1   name               277248 non-null  object 
 2   review_count       277248 non-null  int64  
 3   elite              277248 non-null  object 
 4   friends            277248 non-null  object 
 5   fans               277248 non-null  int64  
 6   average_stars      277248 non-null  float64
 7   compliment_photos  277248 non-null  int64  
dtypes: float64(1), int64(3), object(4)
memory usage: 19.0+ MB
None


,user_id,name,review_count,elite,friends,fans,average_stars,compliment_photos
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1946
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,3.54,0
29,4ZaqBJqt7laPPs8xfWvr6A,Nina,801,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...","v56_o8iCNYGB2JFVIhOwQA, yFLNHDlCriteboCVKjtpbA...",75,3.41,10
32,NIhcRW6DWvk1JQhDhXwgOQ,Lia,2288,"2006,2007,2008,2009,2010,2011,2012,2013,2014,2...","T1upaPMzuW7pNj74fO1rjA, CP28puvAEimt4ziuGTDaHA...",345,3.69,231
50,IpLRJY4CP3fXtlEd8Y4GFQ,Robyn,518,"2009,2010,2011","hdwDo7CLh9aN_9PckAos4Q, ci-mepWGgsgGT3sFI2mhMg...",35,2.95,6


Reviso duplicados:

In [68]:
yelp_users.duplicated().sum()

18530

Hay 6% filas duplicadas del total; las localizo para confirmar:

In [66]:
duplicados= yelp_users[yelp_users.duplicated()]
duplicados.sort_values("user_id")

,user_id,name,review_count,elite,friends,fans,average_stars,compliment_photos
2017202,---2PmXbF47D870stH1jqA,Susan,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,4.98,0
2038302,---2PmXbF47D870stH1jqA,Susan,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,4.98,0
2080502,---2PmXbF47D870stH1jqA,Susan,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,4.98,0
2059402,---2PmXbF47D870stH1jqA,Susan,36,,"o9QkuHIyxmqbORF7PeAbuw, Q70zvjAa9aawgW7KsNVGDA...",3,4.98,0
2053472,--u09WAjW741FdfkJXxNmg,Uyen,263,"2017,2018,2019,20,20,2021","AceePBx22JJXduQbqZpwCg, n4t7lNM91jEHpZid_TEQQw...",68,3.98,105
...,...,...,...,...,...,...,...,...
1999073,zx2NykkcJd1vdOgoS_ZhjQ,Marek,50,,"2psTfvvJyu4L0zVOkw5HBg, nk-wJoOgKPhm7vK0ncUmqA...",0,3.75,0
1987973,zx2NykkcJd1vdOgoS_ZhjQ,Marek,50,,"2psTfvvJyu4L0zVOkw5HBg, nk-wJoOgKPhm7vK0ncUmqA...",0,3.75,0
2095530,zxqDna6zonmT7wI0KDJTtA,Real Shopper,101,,"gKzMIpyPLOkZB3jeA78BGw, 2tUaupnxgEIqKfVSLW9wUQ...",0,3.76,0
2101359,zyguadoKVwAIUOgXmZnM9w,John,161,"2015,2016,2017,2018","SS1z5kWpPm112Un6KFGdQA, Op4fA8FfCLgclffVKMFVTA...",10,3.90,4


Ya que toda la fila está duplicada, las elimino:

In [69]:
yelp_users.drop_duplicates(inplace=True)

# Reviso nuevamente los duplicados:
yelp_users.duplicated().sum()


C:\Users\Usuario\AppData\Local\Temp\ipykernel_3328\4090112948.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_users.drop_duplicates(inplace=True)


0

No hay más duplicados. Ahora, es necesario imputar los "vacíos" de la columna *elite* por "no elite", para aquellos usuario que nunca han sido usuarios elite:

In [75]:
# Reemplazo los registros vacíos "", por "No elite":
yelp_users["elite"] = yelp_users["elite"].replace("", "No elite")

yelp_users.head(2)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3328\1614192359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yelp_users["elite"] = yelp_users["elite"].replace("", "No elite")


,user_id,name,review_count,elite,friends,fans,average_stars,compliment_photos
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1946
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,No elite,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,3.54,0


Reviso la imputación:

In [76]:
empty_elite_mask = yelp_users['elite'].isnull() | (yelp_users['elite'] == '')
yelp_users[empty_elite_mask]

,user_id,name,review_count,elite,friends,fans,average_stars,compliment_photos


Finalmente, reindexo:

In [77]:
yelp_users.reset_index(drop=True, inplace=True)

# Reviso:
yelp_users.head(2)

,user_id,name,review_count,elite,friends,fans,average_stars,compliment_photos
0,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1946
1,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,No elite,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,3.54,0


### 5. FUNCIÓN process_yelp_users():

Ya todas las transformationes están hechas. Ahora, se crear una función que automatice estos cambios:

In [246]:
def process_yelp_users(path, df_reviews=None):
    
    # Defino el df que luego se utilizará para un filtro:
    if df_reviews==None:
        df_reviews= pd.read_csv('POST_ETL_DATASETS/Yelp_Data/yelp_reviews.csv')
        
# EXTRACCIÓN
    # Obtengo el arhivo:
    parquet_file = pq.ParquetFile(path)
    
    # Seleccionar las columnas de interés:
    columns = ['user_id', 'name', 'review_count', 'elite', 'fans',
       'average_stars', 'compliment_photos']

    # Lista para almacenar los DataFrames de cada lote
    batch_dfs = []

    for batch in parquet_file.iter_batches(batch_size=100000, columns=columns):
        # Convertir el lote en un DataFrame de Pandas
        batch_df = batch.to_pandas()
        
# TRANSFORMACIÓN        
        # Obtengo en una lista los user_id únicos de df_reviews:
        lista_user_id= list(df_reviews['user_id'].unique())
        
        # Filtro los usuarios a partir de la lista generada:
        batch_df = batch_df[batch_df['user_id'].isin(lista_user_id)]    
        
        # Agregar el DataFrame del lote a la lista
        batch_dfs.append(batch_df)

    # Concatenar todos los DataFrames de los lotes en uno solo
    df_concat = pd.concat(batch_dfs)

    # Borrar duplicados:
    df_concat.drop_duplicates(inplace=True)
    
    # Imputar valores vacíos en la columna "elite":
    df_concat["elite"] = df_concat["elite"].replace("", "No elite")
    
    # Reindexar:
    df_concat.reset_index(drop=True, inplace=True)

# CARGA    
    df_concat.to_csv('POST_ETL_DATASETS/Yelp_Data/yelp_users.csv', index=False, encoding="utf-8")
    print("Archivo csv guardado con éxito.")
    
    return df_concat

Llamo a la función:

In [247]:
path= 'user.parquet'

yelp_users = process_yelp_users(path)

Archivo csv guardado con éxito.


In [248]:
print(yelp_users.info())
yelp_users

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223488 entries, 0 to 223487
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   user_id            223488 non-null  object 
 1   name               223488 non-null  object 
 2   review_count       223488 non-null  int64  
 3   elite              223488 non-null  object 
 4   fans               223488 non-null  int64  
 5   average_stars      223488 non-null  float64
 6   compliment_photos  223488 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 11.9+ MB
None


,user_id,name,review_count,elite,fans,average_stars,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007,267,3.91,180
1,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,No elite,1,3.54,0
2,4ZaqBJqt7laPPs8xfWvr6A,Nina,801,"2010,2011,2012,2013,2014,2015,2016,2017,2018,2...",75,3.41,10
3,rppTTi-kfF8-qyiArNemag,Helen,460,"2010,2012,2013,2014,2015,2016,2017,2018",49,3.33,25
4,K7thO1n-vZ9PFYiC7nTR2w,Yelper,1554,"2008,2011,2012,2013,2014",326,3.68,105
...,...,...,...,...,...,...,...
223483,_K4X0IBlwDSnClxOrOdxVw,Sarah,14,No elite,0,4.07,0
223484,Pt3u_iZV4Lo--4yfYsBhGg,Jordan,5,No elite,0,2.60,0
223485,ez5L6HxVNNM3nsAvWDVWeQ,Nicole,1,No elite,0,1.00,0
223486,Q5SlTMOwyHq4PIu7Ev-GVg,John,4,No elite,0,5.00,0


#### Ya todos los datos fueron transformados de acuerdo a los objetivos del proyecto y se encuentran listos para ser analizados y crear el MER a partir de ellos.